In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import cyipopt
from pybullet_suite import *
from panda_collision import *
from signed_distance import *
from pathlib import Path

pybullet build time: May 20 2022 19:44:17


In [2]:
mesh_path = Path(PANDA_URDF).parent/"meshes/collision"
link_names = link_names = [
    'link1', 'link2', 'link3', 
    'link4', 'link5', 'link6', 'link7', 'hand', 
    'finger', 'finger'
] # 'link0', 

In [3]:
fk = get_fk_fn(PANDA_URDF)
pc = get_pointclouds(mesh_path, link_names)

Robot name: panda


only got 77/100 samples!


In [4]:
bw = BulletWorld(gui=True)
sm = BulletSceneMaker(bw)
panda = bw.load_robot(Panda, "panda")

argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3090/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 515.105.01
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 515.105.01
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3090/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [5]:
obs = sm.create_sphere("obstacle", 0.3, 0.1, Pose.identity(), [1,0,0,0.3])
obs.set_base_pose(Pose(trans=[0.6,0, 0.6]))

In [6]:
obs_center = jnp.array([0.6, 0, 0.6])
obs_r = 0.3

In [7]:
# env
offset = SE3.from_translation(jnp.array([0., 0, -1]))
half_extents = jnp.array([10., 10, 1])
ground = box(offset, half_extents)
circ = circle(obs_center, obs_r)
env = EnvSDF((circ, ground))

In [8]:
def robot_penet(q):
    tfs = fk(q)[1:]
    points = pc.apply_transforms(tfs)
    return env.penetration_sum(points, pc.num_points, 0.02)

In [50]:
panda.set_joint_angles(panda.joint_central)
q = jnp.array(panda.get_joint_angles())

In [10]:
tfs = fk(q)[1:]
points = pc.apply_transforms(tfs)

In [27]:
q_hess != q_hess.T

Array([[False,  True,  True,  True,  True,  True, False],
       [ True, False,  True,  True,  True,  True, False],
       [ True,  True, False,  True,  True,  True,  True],
       [ True,  True,  True, False,  True, False, False],
       [ True,  True,  True,  True, False,  True,  True],
       [ True,  True,  True, False,  True, False, False],
       [False, False,  True, False,  True, False, False]], dtype=bool)

In [36]:
eigs, v = jnp.linalg.eigh(q_hess)

In [65]:
def hessian(q):
    # def non_pd(H):
        
    #     return Hhat
    # def normal(H):
    #     return H
    H = jax.hessian(robot_penet)(q)
    # for mu in [0, 1, 10, 100, 1000]:
    #     Hhat = H + jnp.eye(7)*mu
    #     cond = jnp.isnan(jnp.linalg.cholesky(H)).any()
        #eig, v = jnp.linalg.eigh(H)
        #Hhat = v@jnp.diag(jnp.maximum(eig, 1e-8))@v.T 
    #cond = jnp.isnan(jnp.linalg.cholesky(H)).any()
    return H #jax.lax.cond(cond, non_pd, normal, H)

In [51]:
H = jax.hessian(robot_penet)(q)

In [66]:
jax.jit(hessian)(q)

Array([[-1.6600764e+03, -7.2850647e+01, -1.6600764e+03, -1.9539661e+02,
        -2.8710504e+02, -1.5986530e+02,  6.7612371e+00],
       [-7.2850311e+01, -1.8518026e+03, -7.2990356e+01,  1.2865939e+03,
        -1.6280579e+01,  1.9237988e+02,  1.0281770e+00],
       [-1.6600763e+03, -7.2990494e+01, -1.6600763e+03, -1.9539662e+02,
        -2.8710504e+02, -1.5986530e+02,  6.7612376e+00],
       [-1.9539670e+02,  1.2865935e+03, -1.9539661e+02, -1.1534746e+03,
        -4.2552608e+01, -2.9699539e+02,  2.6679816e+00],
       [-2.8710498e+02, -1.6280638e+01, -2.8710501e+02, -4.2552635e+01,
        -6.2922955e+01, -3.5036606e+01,  1.4818200e+00],
       [-1.5986526e+02,  1.9237976e+02, -1.5986530e+02, -2.9699542e+02,
        -3.5036583e+01, -1.3239532e+02,  2.2010932e+00],
       [ 6.7612343e+00,  1.0281868e+00,  6.7612495e+00,  2.6679807e+00,
         1.4818192e+00,  2.2010932e+00, -1.3752937e-02]], dtype=float32)

In [67]:
%timeit jax.jit(hessian)(q)

894 µs ± 5.33 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
import time
print(robot_penet(q))
for i in range(100):
    q_grad = jax.grad(robot_penet)(q)
    if jnp.abs(q_grad).sum() < 1e-3:
        break
    q_hess = jax.hessian(robot_penet)(q)
    eig, v = jnp.linalg.eigh(q_hess)
    hess = v@jnp.diag(jnp.maximum(eig, 1e-6))@v.T
    
    steepest_descent_step = -0.0002 * q_grad
    newton_step = - jnp.linalg.inv(hess)@q_grad
    
    q += 0.1 * newton_step/jnp.linalg.norm(newton_step) #steepest_descent_step
    #q += 0.05 * steepest_descent_step/jnp.linalg.norm(steepest_descent_step)
    print(robot_penet(q))
    panda.set_joint_angles(q)
    time.sleep(0.1)
    
#q_hess = jax.hessian(robot_penet)(q)




114.25072
88.01356
52.158867
20.878067
2.8800416
0.00046189502
0.0


In [123]:
jnp.linalg.pinv(q_hess) @ q_grad

Array([-0.00559927, -0.12688619,  0.00681886, -0.10829116,  0.04457071,
        0.11354677, -0.00119272], dtype=float32)

In [205]:
q_hess = jax.hessian(robot_penet)(q)
#q_hess = (q_hess + q_hess.T - jnp.diag(q_hess.diagonal()))/2

In [206]:
jnp.linalg.eigh(q_hess)

(Array([-4.3890942e+03, -2.3425283e+03, -1.9463054e+02, -7.4763632e+00,
        -7.1730804e+00,  1.6137561e-02,  2.7558024e+00], dtype=float32),
 Array([[ 6.92653120e-01, -9.97188464e-02, -2.77884360e-02,
         -3.07605803e-01,  1.13058120e-01,  1.56645267e-03,
          6.34121358e-01],
        [ 1.70451656e-01,  7.58472860e-01,  5.72770596e-01,
         -1.78953618e-01,  1.15394108e-01, -9.75990315e-06,
         -1.49193421e-01],
        [ 6.91758394e-01, -1.07174851e-01, -9.55552906e-02,
          2.43578091e-01, -2.57364124e-01, -1.59720378e-03,
         -6.12604260e-01],
        [-2.75562871e-02, -6.22762024e-01,  6.13084972e-01,
         -3.50754499e-01,  2.22701862e-01, -8.80559615e-04,
         -2.50817090e-01],
        [ 1.08642295e-01, -1.93128120e-02, -3.09155346e-03,
          5.48825741e-01,  8.28588128e-01,  6.00249274e-03,
         -3.35773639e-03],
        [ 9.24866367e-03, -1.22814424e-01,  5.34919083e-01,
          6.24292016e-01, -4.14131433e-01,  6.40763855e-03,


In [91]:
panda.set_joint_angles(q)

In [85]:
jnp.linalg.inv(q_hess) @ q_grad

Array([ 6.9114387e-01, -6.4720487e-04, -6.6610301e-01,  4.3964468e-02,
       -2.1038839e-01, -1.6984457e-01,  8.0105364e-01], dtype=float32)

In [73]:
jnp.linalg.inv(H)

Array([[ 3.18870869e+01, -6.51701748e-01, -3.19050426e+01,
        -1.14817071e+00,  1.92376524e-01,  1.66336489e+00,
         5.87297916e+00],
       [-6.51634872e-01,  4.21889636e-05,  6.51636004e-01,
         7.43030178e-05, -1.23769832e-05, -1.07663844e-04,
        -3.75142699e-04],
       [-3.19050312e+01,  6.51702702e-01,  3.19201584e+01,
         1.14810777e+00, -1.79550827e-01, -1.66327310e+00,
        -5.87993622e+00],
       [-1.14804983e+00,  7.42477714e-05,  1.14798737e+00,
        -1.54743041e-03, -2.09151045e-03,  2.25879368e-03,
        -1.80995792e-01],
       [ 1.92324072e-01, -1.16367682e-05, -1.79498419e-01,
        -2.09023338e-03, -7.35416785e-02,  1.40806856e-02,
         5.46473801e-01],
       [ 1.66320324e+00, -1.07810854e-04, -1.66311216e+00,
         2.25839159e-03,  1.40825622e-02, -5.37359901e-03,
         1.06255174e+00],
       [ 5.87458944e+00, -4.17425937e-04, -5.88154793e+00,
        -1.81070581e-01,  5.46487153e-01,  1.06265569e+00,
         7.6665809

In [ ]:
panda.set_

In [27]:
q_grad

Array([-3.4596138e+00,  1.1593906e+02, -3.4596138e+00, -7.3047043e+01,
       -7.5821018e-01, -8.4767208e+00, -7.8457743e-03], dtype=float32)

In [18]:
joints = jnp.array([panda.get_random_config() for n in range(10)])

In [24]:
%timeit robot_penet_path(joints)

105 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
%timeit robot_penet(jnp.ones(7))

262 µs ± 252 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
pc.apply_transforms(fk(jnp.zeros(7)))

Array([[ 6.3396730e-02, -1.3823391e-02,  5.6593366e-02],
       [-1.4391968e-01, -7.1363188e-03,  2.4019109e-02],
       [-1.3907564e-01,  4.7267664e-02,  1.7710578e-07],
       ...,
       [ 6.0018450e-02,  2.9043026e-02,  8.3249056e-01],
       [ 6.5231413e-02,  3.3720437e-02,  8.2052982e-01],
       [ 4.5467369e-02,  4.7129169e-02,  8.4266722e-01]], dtype=float32)